In [1]:
#https://www.youtube.com/watch?v=JrtuWJYrmSM

In [2]:
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D
import matplotlib.pyplot as plt

In [3]:
# img1 = cv2.imread("G:\\Project-Kratom\\img\\train\\Boiled_leaves\\LINE_ALBUM_ใบต้ม new_220806_0.jpg")
# img1 = cv2.cvtColor(img1 , cv2.COLOR_BGR2RGB)

# img2 = cv2.imread(r'G:\Project-Kratom\img\train\Boiled_leaves\LINE_ALBUM_ใบต้ม new_220806_12.jpg')
# img2 = cv2.cvtColor(img2 , cv2.COLOR_BGR2RGB)

# plt.figure(figsize=(8,4))
# plt.subplot(121),plt.imshow(img1)
# plt.subplot(122),plt.imshow(img2)

In [4]:
width = 128
num_classes = 8
trainpath = r'G:\Project-Kratom\img\train'
testpath = r'G:\Project-Kratom\img\test'
trainImg = [trainpath + "\\"+ f for f in listdir(trainpath)]
testImg = [testpath + "\\"+ f for f in listdir(testpath)]

In [5]:
def img2data(path):
    rawImgs = []
    labels = []

    for imagePath in (path):
        for item in tqdm(listdir(imagePath)):
            file = join(imagePath, item)
            
            if file[-1] =='g':
                img = cv2.imread(file , cv2.COLOR_BGR2RGB)
                img = cv2.resize(img ,(width,width))
                rawImgs.append(img)

                name =  imagePath.split('\\')
                l = name[len(name) - 1]
            #['Boiled_leaves', 'Green_stalk_GradeA', 'Green_stalk_GradeB', 'Green_stalk_GradeC', 'JUMBO', 'Red_stalk_GradeA', 'Red_stalk_GradeB', 'Red_stalk_GradeC']
            if l == 'Boiled_leaves':
                labels.append([1,0,0,0,0,0,0,0])         
            elif l == 'Green_stalk_GradeA':
                labels.append([0,1,0,0,0,0,0,0])  
            elif l == 'Green_stalk_GradeB':
                labels.append([0,0,1,0,0,0,0,0])
            elif l == 'Green_stalk_GradeC':
                labels.append([0,0,0,1,0,0,0,0])
            elif l == 'JUMBO':
                labels.append([0,0,0,0,1,0,0,0])
            elif l == 'Red_stalk_GradeA':
                labels.append([0,0,0,0,0,1,0,0])
            elif l == 'Red_stalk_GradeB':
                labels.append([0,0,0,0,0,0,1,0])
            elif l == 'Red_stalk_GradeC':
                labels.append([0,0,0,0,0,0,0,1])
    return rawImgs, labels

In [ ]:
x_train, y_train = img2data(trainImg)
x_test, y_test = img2data(testImg)

In [7]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
x_train.shape,y_train.shape,x_test.shape, y_test.shape

((1728, 128, 128, 3), (1728, 8), (547, 128, 128, 3), (547, 8))

In [9]:
model0 = Sequential([
        Conv2D(128, (3,3), activation='relu', input_shape=(width, width, 3)),
        MaxPool2D(2), #ทำให้ข้อมูลเล็กลง เพื่อ?
        #Dropout(0.25), #ลดความฉลาด เพื่อลด over fitting
        Conv2D(128,(3,3) , activation='relu'),
        MaxPool2D(pool_size=(2,2 )),
        Dense(16),
        Flatten(),

        Dense(num_classes, activation='softmax') #softmax for one hot . . # sigmoid for 0/1
    ])

In [ ]:
model0.summary()

In [11]:
model0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics= ['accuracy'])
batch_size = 32 
epochs = 20

In [ ]:
history = model0.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.subplot(121),
plt.title('model accuracy');plt.ylabel('accuracy');plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.plot(history.history['accuracy']);plt.plot(history.history['val_accuracy'])

plt.subplot(122)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.plot(history.history['loss']);plt.plot(history.history['val_loss'])

In [14]:
testpath = r'G:\Project-Kratom\img\test'
testImg = [testpath + "\\"+ f for f in listdir(testpath)]
for imagePath in (testImg):
    for i,item in enumerate(listdir(imagePath)):
        file = join(imagePath, item)
        # print(file)

In [ ]:
testpath = r'G:\Project-Kratom\img\test'
testImg = [testpath + "\\"+ f for f in listdir(testpath)]
rimg = []
for imagePath in (testImg):
    j = 0
    for i,item in enumerate(listdir(imagePath)):
        if j == 3:
            break
        file = join(imagePath, item)
        if file[-1] =='g':
        #   print(file)
          imgori = cv2.imread(file)
          imgori = cv2.cvtColor(imgori , cv2.COLOR_BGR2RGB)
          img = cv2.resize(imgori ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,128,128,3))
          predict = model0.predict(rimg)
          #['Boiled_leaves', 'Green_stalk_GradeA', 'Green_stalk_GradeB', 'Green_stalk_GradeC', 'JUMBO', 'Red_stalk_GradeA', 'Red_stalk_GradeB', 'Red_stalk_GradeC']
          label = ['Boiled_leaves','Green_stalk_GradeA','Green_stalk_GradeB','Green_stalk_GradeC','JUMBO','Red_stalk_GradeA','Red_stalk_GradeB','Red_stalk_GradeC'] 
          result = label[np.argmax(predict)]
          print(predict[0]*100)
          plt.title(imagePath)
          cv2.putText(imgori, str(result), (25,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2 )
          plt.imshow(imgori)
          plt.show()
        j += 1

In [16]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                                   include_top=False,
                                                   weights='imagenet')
base_model.trainable = False
num_classes = 8

model = Sequential([
        base_model,
        Conv2D(128, 3, activation='relu'),
        Dense(16),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])

In [ ]:
model.summary()

In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics= ['accuracy'])
batch_size = 32
epochs = 20

In [ ]:
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [61]:
clist = []
wlist = []

In [ ]:
testpath = r'G:\Project-Kratom\img\mini valid'
testImg = [testpath + "\\"+ f for f in listdir(testpath)]
rimg = []
for imagePath in (testImg):
    # j = 0
    for i,item in enumerate(listdir(imagePath)):
        # if j == 1:
            # break
        name = ""
        file = join(imagePath, item)
        if file[-1] =='g':
        #   print(file)
          imgori = cv2.imread(file)
          imgori = cv2.cvtColor(imgori , cv2.COLOR_BGR2RGB)
          img = cv2.resize(imgori ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,128,128,3))
          predict = model.predict(rimg)
          #['Boiled_leaves', 'Green_stalk_GradeA', 'Green_stalk_GradeB', 'Green_stalk_GradeC', 'JUMBO', 'Red_stalk_GradeA', 'Red_stalk_GradeB', 'Red_stalk_GradeC']
          label = ['Boiled_leaves','Green_stalk_GradeA','Green_stalk_GradeB','Green_stalk_GradeC','JUMBO','Red_stalk_GradeA','Red_stalk_GradeB','Red_stalk_GradeC'] 
          result = label[np.argmax(predict)]
          name = result
          print(predict)
          plt.title(imagePath)
          cv2.putText(imgori, str(result), (25,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2 )
          plt.imshow(imgori)
          plt.show()
        # j += 1
    


In [ ]:
model.save("model.h5")

In [68]:
clist

[0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0.7647058823529411,
 0,
 0.25,
 0.7272727272727273,
 0.8333333333333334,
 0,
 1.0,
 1.0,
 1.0]

In [69]:
wlist

[0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0.23529411764705882,
 0,
 0.75,
 0.2727272727272727,
 0.16666666666666666,
 0,
 0.0,
 0.0,
 0.0]

In [ ]:
import keras
modelLoad = keras.models.load_model("model.h5")

In [ ]:
modelLoad.evaluate(x_test, y_test)

18/18 [==============================] - 1s 20ms/step - loss: 0.0404 - accuracy: 0.9927


[0.04039520397782326, 0.9926874041557312]

In [ ]:
perdic = modelLoad.predict([x_test])

18/18 [==============================] - 1s 19ms/step


In [ ]:
perdic[543]

array([5.6950137e-07, 2.5108093e-05, 1.0357776e-04, 1.1919413e-05,
       1.0775048e-03, 1.8368687e-02, 1.3381811e-03, 9.7907442e-01],
      dtype=float32)

In [81]:
# 547
n = 0
per = []
for p in perdic[345]:
    n += p
    per.append(p*100)
n

0.9999999848302181

In [82]:
per

[2.5921302437836857e-06,
 0.01759473670972511,
 0.003631034996942617,
 0.00013385432566792588,
 99.83914494514465,
 0.009441966540180147,
 0.11612167581915855,
 0.01392767735524103]